## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**：但博之


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:
![hw4_1](/fig/hw4_1.jpg)

## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

#### 问题(1)：
证明：
要证明每个最小生成树（MST）都是瓶颈生成树（BST），我们采用反证法：

1. ​假设前提​：
   - 设T是一个最小生成树（MST）
   - 设T的最大边权重为$w_{max}$

2. ​反证假设​：
   - 假设存在另一个生成树T'，其最大边权重$w'_{max} < w_{max}$

3. ​矛盾推导​：
   - 由于T'的所有边权重都不超过$w'_{max}$，而$w'_{max} < w_{max}$
   - 这意味着T'的总权重必然小于T的总权重（因为所有边权重都严格小于T的最大边权重）
   - 这与T是最小生成树的定义矛盾

4. ​结论​：
   - 因此，不存在这样的生成树T'
   - 即MST的最大边权重是所有生成树中最小的
   - 根据定义，MST必为BST

#### 问题(2)：
idea： 

判断是否存在最大边权不超过B的瓶颈生成树 
**（1）算法设计：**  
步骤：
   - 移除图中所有权重大于 b的边。
   - 检查剩余图是否连通。若连通，则存在符合条件的 BST。

**（2）时间复杂度：**  $O(V+E)$。

In [1]:
def has_bottleneck(n, edges, b):
    parent = list(range(n))
    def find(u):
        while parent[u] != u:
            parent[u] = parent[parent[u]]
            u = parent[u]
        return u
    
    def union(u, v):
        pu, pv = find(u), find(v)
        if pu != pv:
            parent[pv] = pu
    
    for u, v, w in edges:
        if w <= b:
            union(u, v)
    
    root = find(0)
    return all(find(i) == root for i in range(n))

# 测试
n = 3
edges = [(0, 1, 1), (1, 2, 2), (0, 2, 3)]
print(has_bottleneck(n, edges, 2))
print(has_bottleneck(n, edges, 1))

True
False


#### 问题(3)：
idea： 

找到瓶颈生成树 
**（1）算法设计：**  
步骤：
   - 使用二分法找到最小b，使得移除权重大于b的边后图仍连通。
   - 在权值不超过b的边集中构造生成树）。

**（2）时间复杂度：**  
- 二分法需检查 $O(logW)$ 次（$W$为最大边权重）。
- 每次检查连通性为 $O(V+E)$。
- 总时间复杂度：$O((V+E)logW)$。

In [2]:
def find_min_b(n, edges):
    left = min(w for _, _, w in edges)
    right = max(w for _, _, w in edges)
    ans = right
    while left <= right:
        mid = (left + right) // 2
        if has_bottleneck(n, edges, mid):
            ans = mid
            right = mid - 1
        else:
            left = mid + 1
    return ans

def construct_bst(n, edges, b):
    adj = [[] for _ in range(n)]
    for u, v, w in edges:
        if w <= b:
            adj[u].append(v)
            adj[v].append(u)
    # BFS 构造生成树
    visited = [False] * n
    tree = []
    from collections import deque
    q = deque([0])
    visited[0] = True
    while q:
        u = q.popleft()
        for v in adj[u]:
            if not visited[v]:
                visited[v] = True
                tree.append((u, v))
                q.append(v)
    return tree if len(tree) == n-1 else None

# 测试
min_b = find_min_b(n, edges)
print("最小瓶颈值:", min_b) 

bst_edges = construct_bst(n, edges, min_b)
print("瓶颈生成树边集:", bst_edges)

最小瓶颈值: 2
瓶颈生成树边集: [(0, 1), (1, 2)]


## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。


本题参考了deepseek大模型

idea：  
**（1）算法设计：**  
步骤：
1. 计算初始最短路径
使用Floyd-Warshall算法计算原始图中任意两个城市之间的最短路径距离矩阵D，其中D[i][j]表示城市i到城市j的最短路径长度。
2. 评估新道路影响
对于每条候选新道路(u', v')：
    - 将其添加到原图中，形成新图
    - 重新运行Floyd-Warshall算法计算新图的最短路径矩阵D'
    - 计算s→t的最短路径减少量：Δ = D[s][t] - D'[s][t]

3. 选择最优新道路
遍历所有候选道路，选择使Δ最大的道路(u*, v*)作为最优解。

**（2）算法正确性解释：**  
- ​步骤1​：Floyd-Warshall算法能够正确计算所有城市对的最短路径，时间复杂度为O(n³)
- ​步骤2​：通过重新计算新图的最短路径，可准确评估添加某条道路后对s→t路径的影响
- ​步骤3​：遍历所有候选道路，确保找到使s→t路径减少量最大的解

**（2）时间复杂度：**  
- Floyd-Warshall算法：$O(n³)$
- 更新最短路径矩阵：对每条候选道路($u'$, $v'$) ∈ $E'$，需$O(n³)$
- 总时间复杂度：$O(|E'|·n³)$，其中$|E'|$为候选道路数量

In [3]:
import numpy as np

def floyd_warshall(graph):
    n = len(graph)
    dist = np.copy(graph)
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
    return dist

def find_best_new_road(V, E, l, E_prime, l_prime, s, t):
    graph = np.full((len(V), len(V)), np.inf)
    for i in range(len(V)):
        graph[i][i] = 0
    
    for (u, v) in E:
        graph[u][v] = l[(u, v)]

    initial_dist = floyd_warshall(graph)
    original_shortest = initial_dist[s][t]
    
    best_road = None
    max_reduction = 0
    
    for (u_prime, v_prime) in E_prime:
        new_graph = np.copy(graph)
        new_graph[u_prime][v_prime] = l_prime[(u_prime, v_prime)]
        new_dist = floyd_warshall(new_graph)
        new_shortest = new_dist[s][t]
        reduction = original_shortest - new_shortest
        if reduction > max_reduction:
            max_reduction = reduction
            best_road = (u_prime, v_prime)
    
    return best_road, max_reduction

# 测试
V = [0, 1, 2, 3]
E = [(0, 1), (1, 2), (2, 3)]
l = {(0, 1): 1, (1, 2): 1, (2, 3): 1}
E_prime = [(0, 3)]
l_prime = {(0, 3): 2}
s, t = 0, 3  # 指定源点s和终点t

best_road, max_reduction = find_best_new_road(V, E, l, E_prime, l_prime, s, t)
print("最佳新道路:", best_road)
print("最大距离减少量:", max_reduction)

最佳新道路: (0, 3)
最大距离减少量: 1.0


## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图2. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

本题参考了deepseek大模型

idea：  
#### 问题(1)：

顶点和边容量网络简化为仅有边容量的网络  
**（1）算法设计：**  
1. ​顶点拆分法​：
   - 将每个顶点$v$拆分为两个顶点$v_{in}$和$v_{out}$
   - 所有进入$v$的边改为进入$v_{in}$
   - 所有离开$v$的边改为从$v_{out}$离开
   - 在$v_{in}$和$v_{out}$之间添加一条边，容量为原顶点$v$的容量

2. ​边保留处理​：
   - 原边容量保持不变
   - 原边$(u,v)$将转换为$u_{out}→v_{in}$，容量与原边相同

**（2）时间复杂度：**  构造新网络的时间复杂度为$O(V+E)$，其中$V$是原顶点数，$E$是原边数。

#### 问题(2)：

逃离问题的高效算法
**（1）算法设计：**  
1. ​网络构建​：
   - 将每个网格顶点$(i,j)$拆分为$in(i,j)$和$out(i,j)$
   - 边$in(i,j)→out(i,j)$的容量为1（顶点容量）
   - 相邻顶点间的边转换为$out(i,j)→in(k,l)$，容量为1（边容量）
   - 超级源点$S$连接到每个起点$(x_i,y_i)$的$in$节点，容量为1
   - 所有边界顶点的$out$节点连接到超级汇点$T$，容量为1

2. ​最大流计*​：
   - 使用Dinic算法计算从$S$到$T$的最大流
   - 若最大流等于起点数$m$，则存在逃离路径

**（2）时间复杂度：**  顶点数$O(n^2)$，边数$O(n^2)$，Dinic算法的时间复杂度为$O(E\sqrt{V}) = O(n^3)$

In [4]:
from collections import deque

class Edge:
    def __init__(self, to, rev, capacity):
        self.to = to
        self.rev = rev
        self.capacity = capacity

class Dinic:
    def __init__(self, n):
        self.size = n
        self.graph = [[] for _ in range(n)]
    
    def add_edge(self, fr, to, cap):
        forward = Edge(to, len(self.graph[to]), cap)
        backward = Edge(fr, len(self.graph[fr]), 0)
        self.graph[fr].append(forward)
        self.graph[to].append(backward)
    
    def bfs_level(self, s, t, level):
        q = deque([s])
        level[:] = [-1] * self.size
        level[s] = 0
        while q:
            v = q.popleft()
            for edge in self.graph[v]:
                if edge.capacity > 0 and level[edge.to] == -1:
                    level[edge.to] = level[v] + 1
                    q.append(edge.to)
                    if edge.to == t:
                        return
    
    def dfs_flow(self, v, t, flow, level, ptr):
        if v == t:
            return flow
        while ptr[v] < len(self.graph[v]):
            edge = self.graph[v][ptr[v]]
            if edge.capacity > 0 and level[v] < level[edge.to]:
                min_flow = min(flow, edge.capacity)
                result = self.dfs_flow(edge.to, t, min_flow, level, ptr)
                if result > 0:
                    edge.capacity -= result
                    self.graph[edge.to][edge.rev].capacity += result
                    return result
            ptr[v] += 1
        return 0
    
    def max_flow(self, s, t):
        flow = 0
        level = [-1] * self.size
        while True:
            self.bfs_level(s, t, level)
            if level[t] == -1:
                return flow
            ptr = [0] * self.size
            while True:
                f = self.dfs_flow(s, t, float('inf'), level, ptr)
                if f == 0:
                    break
                flow += f
            level = [-1] * self.size
        return flow

def build_escape_network(n, starts):
    total_nodes = 2 * n * n + 2 
    s = 2 * n * n
    t = s + 1
    dinic = Dinic(total_nodes)
    
    # 添加顶点拆分边
    for i in range(n):
        for j in range(n):
            in_node = i * n + j
            out_node = in_node + n * n
            dinic.add_edge(in_node, out_node, 1)
    
    # 添加相邻边
    dirs = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    for i in range(n):
        for j in range(n):
            out_node = i * n + j + n * n
            for dx, dy in dirs:
                ni, nj = i + dx, j + dy
                if 0 <= ni < n and 0 <= nj < n:
                    neighbor_in = ni * n + nj
                    dinic.add_edge(out_node, neighbor_in, 1)
    
    # 超级源点连接起点
    for (x, y) in starts:
        i, j = x - 1, y - 1  
        in_node = i * n + j
        dinic.add_edge(s, in_node, 1)
    
    # 超级汇点连接边界顶点
    for i in range(n):
        for j in range(n):
            if i == 0 or i == n-1 or j == 0 or j == n-1:
                out_node = i * n + j + n * n
                dinic.add_edge(out_node, t, 1)
    
    return dinic, s, t

def has_escape(n, starts):
    m = len(starts)
    if m == 0:
        return True
    if m > n * n:
        return False
    dinic, s, t = build_escape_network(n, starts)
    return dinic.max_flow(s, t) == m

# 测试
n = 3
starts = [(1, 1), (2, 2)]  
print(has_escape(n, starts))

True
